In [1]:
import os
import numpy as np
import pandas as pd
import swifter
from sklearn.model_selection import train_test_split
import utils
from hep_ml.nnet import MLPClassifier

Using cuDNN version 6021 on context None
Mapped name None to device cuda: GeForce GTX TITAN X (0000:02:00.0)


In [2]:
DATA_PATH = "./data"

In [3]:
train, test = utils.load_data_csv(DATA_PATH, utils.SIMPLE_FEATURE_COLUMNS)

/root/anaconda3/envs/idao_cb/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
func = utils.find_closest_hit_per_station

In [5]:
# dt_train = pd.read_hdf('closest_hits_features.train.filled.p1000.hdf', 'key')
# dt_test  = pd.read_hdf('closest_hits_features.test.filled.p1000.hdf', 'key')

dt_train = pd.read_hdf('data/closest_hits_features.train.filled.m9999.v1.hdf', 'key')
dt_test  = pd.read_hdf('data/closest_hits_features.test.filled.m9999.v1.hdf', 'key')

In [6]:
dt_train.columns = [func.__name__ + '_' + str(c) if str(c).isnumeric() else c for c in dt_train.columns]
dt_test.columns  = [func.__name__ + '_' + str(c) if str(c).isnumeric() else c for c in dt_test.columns]

In [7]:
train = pd.concat([train, dt_train], axis=1, copy=False)
test = pd.concat([test, dt_test], axis=1, copy=False)

In [8]:
train.shape, test.shape

((5445705, 115), (726095, 113))

In [9]:
set(train.columns.tolist()).symmetric_difference(test.columns.tolist())

{'label', 'weight'}

In [10]:
del test

## Новые признаки

In [11]:
NA_VALUE = -9999.0

In [12]:
def fillna(df):
    """
    Пропуски только в 2 и 3 колонках
    """
    
    for axis in 'XYZ':
        cols = ['MatchedHit_{}[{}]'.format(axis, i) for i in range(4)]
        mask = np.isclose(df.loc[:, cols[2]], NA_VALUE)
        mask = mask & ~np.isclose(df.loc[:, cols[3]], NA_VALUE)
        
        df.loc[mask, cols[2]] = df.loc[mask, [cols[1], cols[3]]].mean(axis=1)
    return df

In [13]:
# train, test = map(fillna, [train, test])

In [14]:
train = fillna(train)

### Модифицируем matched_hit_type

In [15]:
def modify_matched_hit_type(df):
    map_type = {2: 1, 0: 0, 1: -1}
    cols = df.columns[df.columns.str.startswith('MatchedHit_TYPE')]
    for col in cols:
        df[col] = df[col].map(map_type)
    return df

In [16]:
# train, test = map(modify_matched_hit_type, [train, test])

In [17]:
train = modify_matched_hit_type(train)

### Вычесть среднее из MatchedHit_Z

In [18]:
matched_hit_z_cols = train.columns[train.columns.str.startswith("MatchedHit_Z")]
matched_hit_z_mean = pd.Series([15270, 16470, 17670, 18870], index=matched_hit_z_cols)
matched_hit_z_mean

MatchedHit_Z[0]    15270
MatchedHit_Z[1]    16470
MatchedHit_Z[2]    17670
MatchedHit_Z[3]    18870
dtype: int64

In [19]:
def make_unbiased_z(df):
    for i in range(4):
        distance = np.zeros(df.shape[0])
    
        col = f"MatchedHit_Z[{i}]"
        mask = np.isclose(df.loc[:, col], NA_VALUE)
        distance[~mask] = df.loc[~mask, col] - matched_hit_z_mean[col]
        distance[mask] = 0
    
        df[col] = distance
        
        df[f"MatchedHit_Z[{i}]_sign"] = np.sign(df[col]).astype(np.int8)
    return df

In [20]:
# train, test = map(make_unbiased_z, [train, test])

In [21]:
train = make_unbiased_z(train)

In [22]:
train.loc[:, train.columns.str.startswith("MatchedHit_Z")].head()

,MatchedHit_Z[0],MatchedHit_Z[1],MatchedHit_Z[2],MatchedHit_Z[3],MatchedHit_Z[0]_sign,MatchedHit_Z[1]_sign,MatchedHit_Z[2]_sign,MatchedHit_Z[3]_sign
0,-146.955,-142.367,-149.174,-140.676,-1,-1,-1,-1
1,-57.402,-49.424,-55.146,-45.570,-1,-1,-1,-1
2,47.953,52.060,-71.700,-63.648,1,1,-1,-1
3,-68.590,-64.445,-71.700,-63.648,-1,-1,-1,-1
4,-64.511,-60.047,-66.979,-58.605,-1,-1,-1,-1


### Расстояние до центра (+)

In [23]:
def compute_polar_distances_matched_hit(df):
    for i in range(4):
        cols = [f"MatchedHit_X[{i}]", f"MatchedHit_Y[{i}]"]
        mask = np.isclose(df.loc[:, cols], NA_VALUE).any(axis=1)
    
        name = f"MatchedHit_R[{i}]"
        df[name] = NA_VALUE
        df.loc[~mask, name] = np.log(1.0 + np.power(df.loc[~mask, cols], 2).sum(axis=1))
        
        name = f"MatchedHit_A[{i}]"
        df[name] = NA_VALUE
        df.loc[~mask, name] = np.arctan2(df.loc[~mask, cols[1]], df.loc[~mask, cols[0]])

    # намеренное переупорядочивание признаков
    cols  = df.columns[df.columns.str.startswith("MatchedHit_R")].tolist()
    cols += df.columns[df.columns.str.startswith("MatchedHit_A")].tolist()
    cols  = df.columns[:-len(cols)].tolist() + cols
    df = df[cols]

    return df

In [24]:
# train, test = map(compute_polar_distances_matched_hit, [train, test])

In [25]:
train = compute_polar_distances_matched_hit(train)

In [26]:
train.loc[:, train.columns.str.startswith("MatchedHit_R")].head()

,MatchedHit_R[0],MatchedHit_R[1],MatchedHit_R[2],MatchedHit_R[3]
0,16.023317,16.206292,16.332808,16.583070
1,15.647796,15.811899,15.934739,16.044511
2,14.788550,14.971064,15.147750,15.280270
3,14.899596,15.022741,15.147750,15.280270
4,14.962988,15.139525,15.316049,15.449411


In [27]:
train.loc[:, train.columns.str.startswith("MatchedHit_A")].head()

,MatchedHit_A[0],MatchedHit_A[1],MatchedHit_A[2],MatchedHit_A[3]
0,2.668808,2.676832,2.672886,2.701153
1,1.270437,1.251195,1.280089,1.319231
2,-2.944427,-2.947037,-3.027307,-3.028216
3,-3.030048,-3.028084,-3.027307,-3.028216
4,2.582489,2.590782,2.601329,2.600883


In [28]:
def replace_na(df):
    df[df == NA_VALUE] = None
    df.fillna(df.mean(), inplace=True)
    return df

In [29]:
# train, test = map(replace_na, [train, test])

In [30]:
train = replace_na(train)

### Дельты и отношения для радиусов (+)

In [31]:
def ratio_radius_matched_hit(df):
    for i, j in zip([0, 1, 2, 3], [1, 2, 3, 0]):
        col_i = f"MatchedHit_R[{i}]"
        col_j = f"MatchedHit_R[{j}]"
        mask = np.isclose(df.loc[:, [col_i, col_j]], NA_VALUE).any(axis=1)

        name = f"MatchedHit_R[{j}/{i}]_delta"
        name_abs = name + '_abs'
        df[name] = df[name_abs] = NA_VALUE
        
        df.loc[~mask, name] = df.loc[~mask, col_j] - df.loc[~mask, col_i]
        df.loc[~mask, name_abs] = np.abs(df.loc[~mask, name])
        
        name = f"MatchedHit_R[{j}/{i}]_frac"
        df[name] = NA_VALUE
        df.loc[~mask, name] = df.loc[~mask, col_j] / df.loc[~mask, col_i]
    return df

In [32]:
# train, test = map(ratio_radius_matched_hit, [train, test])

In [33]:
train = ratio_radius_matched_hit(train)

In [34]:
cols = filter(lambda s: s.startswith("MatchedHit_R") and ("_delta" in s or "_frac" in s), train.columns)
train.loc[:, cols].head()

,MatchedHit_R[1/0]_delta,MatchedHit_R[1/0]_delta_abs,MatchedHit_R[1/0]_frac,MatchedHit_R[2/1]_delta,MatchedHit_R[2/1]_delta_abs,MatchedHit_R[2/1]_frac,MatchedHit_R[3/2]_delta,MatchedHit_R[3/2]_delta_abs,MatchedHit_R[3/2]_frac,MatchedHit_R[0/3]_delta,MatchedHit_R[0/3]_delta_abs,MatchedHit_R[0/3]_frac
0,0.182975,0.182975,1.011419,0.126516,0.126516,1.007807,0.250262,0.250262,1.015323,-0.559754,0.559754,0.966245
1,0.164103,0.164103,1.010487,0.122840,0.122840,1.007769,0.109772,0.109772,1.006889,-0.396714,0.396714,0.975274
2,0.182513,0.182513,1.012342,0.176687,0.176687,1.011802,0.132520,0.132520,1.008748,-0.491720,0.491720,0.967820
3,0.123146,0.123146,1.008265,0.125009,0.125009,1.008321,0.132520,0.132520,1.008748,-0.380674,0.380674,0.975087
4,0.176537,0.176537,1.011798,0.176523,0.176523,1.011660,0.133362,0.133362,1.008707,-0.486422,0.486422,0.968515


### Расстояние между Lextra и MatchedHit (+)

In [35]:
def distance_dist_lextra_matched(df):
    distance_counts = np.zeros(df.shape[0], dtype=int)
    
    distance_se = np.zeros(df.shape[0])
    distance_ae = np.zeros(df.shape[0])
    
    for i in range(4):
        distance = np.zeros(df.shape[0])

        cols = [f"MatchedHit_X[{i}]", f"MatchedHit_Y[{i}]", f"Lextra_X[{i}]", f"Lextra_Y[{i}]"]
        mask = np.isclose(df.loc[:, cols], NA_VALUE).any(axis=1)

        mse = df.loc[~mask, cols[:2]].values - df.loc[~mask, cols[2:]].values
        mse = np.power(mse, 2.0).sum(axis=1)
        
        mae = df.loc[~mask, cols[:2]].values - df.loc[~mask, cols[2:]].values
        mae = np.abs(mae).max(axis=1)

        distance[~mask] = np.log(1.0 + mse)
        distance[mask] = NA_VALUE
        
        distance_se[~mask] += mse
        distance_ae[~mask] = np.maximum(distance_ae[~mask], mae)
        distance_counts[~mask] += 1

        df[f"distance_LextraMatchedHit[{i}]"] = distance

#     distance_se /= distance_counts
#     df[f"distance_LextraMatchedHit_SE"] = distance_se
#     df[f"distance_LextraMatchedHit_AE"] = distance_ae

    return df

In [36]:
# train, test = map(distance_dist_lextra_matched, [train, test])

In [37]:
train = distance_dist_lextra_matched(train)

In [38]:
train.loc[:, train.columns.str.startswith("distance_LextraMatchedHit")].head()

,distance_LextraMatchedHit[0],distance_LextraMatchedHit[1],distance_LextraMatchedHit[2],distance_LextraMatchedHit[3]
0,6.712163,7.512178,7.232589,10.468368
1,7.448349,7.150894,9.267218,11.013328
2,7.192176,7.376499,9.402387,9.477220
3,5.449827,6.342438,6.628810,5.935092
4,8.099670,8.254467,8.448039,8.426182


### Отношение дисперсий

In [39]:
def ratio_mextra_matched(df):    
    for i in range(4):
        cols = [f"MatchedHit_DX[{i}]", f"MatchedHit_DY[{i}]", f"Mextra_DX2[{i}]", f"Mextra_DY2[{i}]"]
        mask = np.isclose(df.loc[:, cols], NA_VALUE).any(axis=1)

        tmpX = df.loc[~mask, cols[::2]]
        tmpY = df.loc[~mask, cols[1::2]]
        
        ratio = tmpX.iloc[:, 0] / np.sqrt(tmpX.iloc[:, 1]) + \
                tmpY.iloc[:, 0] / np.sqrt(tmpY.iloc[:, 1])

        name = f"ratio_MextraMatchedHit[{i}]"
        df[name] = NA_VALUE
        df.loc[~mask, name] = ratio

    return df

In [40]:
# train, test = map(ratio_mextra_matched, [train, test])

In [41]:
train = ratio_mextra_matched(train)

In [42]:
train.loc[:, train.columns.str.startswith("ratio_MextraMatchedHit")].head()

,ratio_MextraMatchedHit[0],ratio_MextraMatchedHit[1],ratio_MextraMatchedHit[2],ratio_MextraMatchedHit[3]
0,8.287385,5.293275,5.687804,4.349901
1,18.965682,12.044753,13.049134,32.661490
2,6.001871,3.893571,4.204257,3.224819
3,12.649516,8.039806,8.694506,6.719570
4,5.487896,3.481615,3.740265,2.870688


### Еще один момент

In [43]:
def momentum(df):
    mask = np.isclose(df.loc[:, ['P', 'PT']], NA_VALUE).any(axis=1)
    df['PO'] = NA_VALUE
    df.loc[~mask, 'PO'] = np.power(df.loc[~mask, 'P'], 2.0) - np.power(df.loc[~mask, 'PT'], 2.0)
    return df

In [44]:
# train, test = map(momentum, [train, test])

In [45]:
train = momentum(train)

In [46]:
train.loc[:, ['P', 'PT', 'PO']].head()

,P,PT,PO
0,12646.817591,1764.925899,1.568270e+08
1,28556.630027,4511.281266,7.951295e+08
2,17491.702634,1062.550086,3.048306e+08
3,37192.868559,4991.074825,1.358399e+09
4,16688.746174,1347.704888,2.766979e+08


### Усредняем дисперсии

In [47]:
def mean_equals(df):
    for n in [0, 1, 2, 3]:
        df[f'Mextra_DXY2[{n}]_mean'] = sum([df[f'Mextra_D{k}2[{n}]'] for k in ('X', 'Y')])/2
        df[f'MatchedHit_DXYZ[{n}]_mean'] = sum([df[f'MatchedHit_D{k}[{n}]'] for k in ('X', 'Y', 'Z')])/3
        df.drop(columns=[f'Mextra_D{k}2[{n}]' for k in ('X', 'Y')], inplace=True)
        df.drop(columns=[f'MatchedHit_D{k}[{n}]' for k in ('X', 'Y', 'Z')], inplace=True)
    return df

In [48]:
# train, test = map(mean_equals, [train, test])

In [49]:
train = mean_equals(train)

### Удаляем лишние колонки

In [50]:
import operator
from functools import reduce

def drop_columns(df):
    mask = [df.columns.str.startswith('MatchedHit_X'),
            df.columns.str.startswith('MatchedHit_Y'),
            df.columns.str.startswith('Lextra_X'),
            df.columns.str.startswith('Lextra_Y'),
            df.columns.str.startswith('FOI_hits_N['),
            df.columns.str.startswith('ndof'),
            df.columns.str.startswith('MatchedHit_DT'),
            df.columns.str.startswith('MatchedHit_T'),
            df.columns.str.endswith('sign'),
            df.columns.str.endswith('abs'),
            df.columns.str.endswith('delta')]
    mask = reduce(operator.or_, mask)
    cols = df.columns[mask].tolist()
    cols += df.columns[df.columns.str.startswith('find_closest_hit_per_station')].tolist()
    
    df.drop(columns=cols, inplace=True)
    return df

In [51]:
# train, test = map(drop_columns, [train, test])

In [52]:
train = drop_columns(train)

## Обучение модели

I know this is incorrect. See it as a low-hanging fruit to beat the baseline. Catboost affirmatively [refused](https://github.com/catboost/catboost/pull/399) to support negative weights. At the same time, its evaluation is [extremly fast](https://catboost.ai/news/best-in-class-inference-and-a-ton-of-speedups), so we are using it as the timing benchmark for Track 2. Feel free to use a patched version with disabled negative weights check.

In [53]:
features = list(filter(lambda s: s not in utils.TRAIN_COLUMNS, train.columns))
features

['ncl[0]',
 'ncl[1]',
 'ncl[2]',
 'ncl[3]',
 'avg_cs[0]',
 'avg_cs[1]',
 'avg_cs[2]',
 'avg_cs[3]',
 'MatchedHit_Z[0]',
 'MatchedHit_Z[1]',
 'MatchedHit_Z[2]',
 'MatchedHit_Z[3]',
 'NShared',
 'FOI_hits_N',
 'PT',
 'P',
 'MatchedHit_R[0]',
 'MatchedHit_R[1]',
 'MatchedHit_R[2]',
 'MatchedHit_R[3]',
 'MatchedHit_A[0]',
 'MatchedHit_A[1]',
 'MatchedHit_A[2]',
 'MatchedHit_A[3]',
 'MatchedHit_R[1/0]_frac',
 'MatchedHit_R[2/1]_frac',
 'MatchedHit_R[3/2]_frac',
 'MatchedHit_R[0/3]_frac',
 'distance_LextraMatchedHit[0]',
 'distance_LextraMatchedHit[1]',
 'distance_LextraMatchedHit[2]',
 'distance_LextraMatchedHit[3]',
 'ratio_MextraMatchedHit[0]',
 'ratio_MextraMatchedHit[1]',
 'ratio_MextraMatchedHit[2]',
 'ratio_MextraMatchedHit[3]',
 'PO',
 'Mextra_DXY2[0]_mean',
 'MatchedHit_DXYZ[0]_mean',
 'Mextra_DXY2[1]_mean',
 'MatchedHit_DXYZ[1]_mean',
 'Mextra_DXY2[2]_mean',
 'MatchedHit_DXYZ[2]_mean',
 'Mextra_DXY2[3]_mean',
 'MatchedHit_DXYZ[3]_mean']

In [54]:
params = {}

for col in features:
    mean, std = train[col].mean(), train[col].std()
    params[col] = (mean, std)
    
    scale_mean_std = lambda x: (x - mean) / std
#     train[col], test[col] = map(scale_mean_std, [train[col], test[col]])
    train[col] = scale_mean_std(train[col])

In [55]:
# Заглушка для scaler

import copy

from sklearn.base import TransformerMixin

class IdentityScaler(TransformerMixin):
    def __init__(self):        
        pass
    
    def fit(self, X, y=None, **fit_params):        
        return self
        
    def transform(self, X, y=None):
        return X
    
    def fit_transform(self, X, y=None, **fit_params):
        return self.transform(X, y)
    
    def get_params(self, deep=True):
        return copy.deepcopy(self.__dict__) if deep else self.__dict__
    
    def set_params(self, **params):
        self.__dict__.update(copy.deepcopy(params))

In [56]:
%%time

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss
from scoring import rejection90

models_g = []
scores_g = []

# kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=4986)
kf = KFold(n_splits=5, shuffle=True, random_state=4987)

np.random.seed(5873)
seeds = np.random.randint(1_000, 10_000, size=5)
print("seeds:", *seeds)

for train_i, valid_i in kf.split(train):
    X_train, w_train, y_train = train.loc[train_i, features], \
        train.loc[train_i, 'weight'], train.loc[train_i, 'label']
    X_valid, w_valid, y_valid = train.loc[valid_i, features], \
        train.loc[valid_i, 'weight'], train.loc[valid_i, 'label']
    
    X_train, w_train, y_train = map(lambda o: o.values, [X_train, w_train, y_train])
    X_valid, w_valid, y_valid = map(lambda o: o.values, [X_valid, w_valid, y_valid])
    
    model = MLPClassifier(layers=[32], epochs=180,
                          scaler=IdentityScaler(), random_state=seeds[len(models_g)])
    model.fit(X_train, y_train, sample_weight=w_train)
    
    y_pred = model.predict_proba(X_valid)[:,1]
    
    score = {
        'log_loss': log_loss(y_valid, y_pred),
        'auc': roc_auc_score(y_valid, y_pred),
        'rejection90': rejection90(y_valid, y_pred, sample_weight=w_valid)
    }
    
    print("log_loss = {log_loss:.6f}; auc = {auc:.6f}; rejection90 = {rejection90:.6f}".format(**score))
    
    models_g.append(model)
    scores_g.append(score)

seeds: 3389 6094 5678 5433 6067
log_loss = 0.417855; auc = 0.816864; rejection90 = 0.736098
log_loss = 0.418969; auc = 0.814543; rejection90 = 0.751751
log_loss = 0.408720; auc = 0.815936; rejection90 = 0.736316
log_loss = 0.421045; auc = 0.812465; rejection90 = 0.750978
log_loss = 0.411237; auc = 0.816379; rejection90 = 0.755377
CPU times: user 3min 33s, sys: 55.5 s, total: 4min 28s
Wall time: 1min 46s


In [57]:
from collections import defaultdict
from functools import reduce
from operator import setitem

score = reduce(lambda d, sc: any(setitem(d, k, d[k] + v) for k, v in sc.items()) or d,
               scores_g, defaultdict(float))

for k, v in score.items():
    score[k] = v / len(scores_g)
    
print("log_loss = {log_loss:.6f}; auc = {auc:.6f}; rejection90 = {rejection90:.6f}".format(**score))

log_loss = 0.415565; auc = 0.815237; rejection90 = 0.746104


In [58]:
score['rejection90']

0.7461040215228469

## Сохраняем модельки

In [59]:
!mkdir -p models

In [60]:
import pickle

pickle.dump(params, open('models/scaler_params.pkl', 'wb'))

for i, model in enumerate(models_g):
    pickle.dump(model, open(f'models/hepmlp_model_{i}.pkl', 'wb'))